In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np 
import pandas as pd 
import os
import cv2
import matplotlib.pyplot as plt

#### 레이블

In [3]:
labels = os.listdir("/content/drive/MyDrive/D.D.D./train")
labels

['Closed', 'yawn', 'Open', 'no_yawn']

#### 이미지 불러와서 저장하기

In [4]:
def get_data(dir_path="/content/drive/MyDrive/D.D.D./train/", 
             face_cas="/content/drive/MyDrive/D.D.D./haarcascade_frontalface_default.xml", 
             eye_cas="/content/drive/MyDrive/D.D.D./haarcascade_eye.xml"):
    labels = ['Closed', 'Open']
    IMG_SIZE = 145
    data = []
    for label in labels:
        path = os.path.join(dir_path, label)
        class_num = labels.index(label)
       # class_num +=2
        print(class_num)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_COLOR)
                resized_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                data.append([resized_array, class_num])
            except Exception as e:
                print(e)
    return data

In [5]:
data_train = get_data()

0
1


In [6]:
new_data = get_data()

0
1


#### 레이블과 이미지 분류

In [7]:
X = []
y = []
for feature, label in new_data:
    X.append(feature)
    y.append(label)

3차원 배열을 1차원 배열로 변경하기 위해 첫 번째 인자에 -1을 넣어줌

In [8]:
X = np.array(X)
X = X.reshape(-1, 145, 145, 3)

In [9]:
from sklearn.preprocessing import LabelBinarizer

label_bin = LabelBinarizer()
y = label_bin.fit_transform(y)

#### 훈련, 검증 데이터 분리

In [10]:
from sklearn.model_selection import train_test_split

seed = 42
test_size = 0.30
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=seed, test_size=test_size)

In [11]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1016, 145, 145, 3)
(436, 145, 145, 3)
(1016, 1)
(436, 1)


numpy.ndarray : "H x W x C" 

*   H : height 높이
*   W : width 너비
*   C : channel 채널 (RGB 컬러의 경우 3, grayscale의 경우 1또는 0)

#### 이미지 파일 텐서로 변환

In [12]:
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torchvision
from torchvision import transforms

https://supermemi.tistory.com/139

In [13]:
from torchvision.transforms import ToTensor, ToPILImage

In [14]:
a = plt.imread("/content/drive/MyDrive/D.D.D./train/Closed/_0.jpg")
a_array = np.array(a)

In [15]:
print(f'RGB  Image shape : {a.shape}')  # 428 x 428 x 3
print(f'RGB  Value Scale : {a.min()}~{a.max()}') # 0~255

RGB  Image shape : (145, 145, 3)
RGB  Value Scale : 0~170


In [16]:
# torchvision.transforms.ToTensor
tf_toTensor = ToTensor() 

In [ ]:
# PIL to Tensor
img_RGB_tensor_from_PIL = tf_toTensor(img_RGB)

print(img_RGB_tensor_from_PIL)
print(img_RGB_tensor_from_PIL.size()) # 3 x 428 x 428
print(img_RGB_tensor_from_PIL.min(), img_RGB_tensor_from_PIL.max()) # 0~1


# Numpy array to Tensor
img_RGB_tensor_from_ndarray = tf_toTensor(img_RGB_array)

print(img_RGB_tensor_from_ndarray)
print(img_RGB_tensor_from_ndarray.size()) # 3 x 428 x 428
print(img_RGB_tensor_from_ndarray.min(), img_RGB_tensor_from_ndarray.max()) # 0~1


# torchvision.transforms.ToPILImage
tf_toPILImage = ToPILImage() 

img_PIL_from_Tensor = tf_toPILImage(img_RGB_tensor_from_PIL)

print(np.array(img_PIL_from_Tensor)) # 0~255
print(np.array(img_PIL_from_Tensor).shape) # 428 x 428 x 3


# Check values between numpy array and tensor
print(np.sum(~(img_RGB_array==np.array(img_PIL_from_Tensor)))) # 0 means all same

Pytorch의 Image tensor : "C x H x W" 의 구조

 

- Channel의 위치가 변경되는 문제가 발생한다.
- 이미지 픽셀 밝기 값은 0 ~ 255의 범위에서 0~1의 범위로 변경된다.

 

이를 해결하기 위하여 torchvision.transforms 모듈을 이용한다.

https://sanghyu.tistory.com/19

In [17]:
tf_toTensor(X_train)

ValueError: ignored

https://cyc1am3n.github.io/2018/09/13/how-to-use-dataset-in-tensorflow.html

#### 모델 훈련하기

In [18]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

*   torch.utils.data.DataLoader : Dataset 을 순회 가능한 객체(iterable)로 감싼다.
*   torch.utils.data.Dataset : Dataset 은 샘플과 정답(label)을 저장한다.



In [19]:
batch_size = 64

# 데이터로더를 생성한다.
train_dataloader = DataLoader(X_train, batch_size=batch_size)
test_dataloader = DataLoader(X_test, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

ValueError: ignored

#### 모델 만들기

- pytorch에서 신경망 모델은 nn.Module을 상속받는 클래스를 생성하여 정의한다.
- __init__ 함수에서 신경망의 계층(layer)들을 정의하고 forward 함수에서 신경망에 데이터를 어떻게 전달할지 지정한다.
- 가능한 경우 GPU로 신경망을 이동시켜 연산을 가속(accelerate)한다.

In [20]:
# 학습에 사용할 CPU나 GPU 장치를 얻는다

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [21]:
# 모델을 정의한다.

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


#### 모델 매개변수 최적화하기

- 손실 함수(loss function)과 옵티마이저(optimizer)가 필요함

In [22]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

각 학습 단계(training loop)에서 모델은 (배치(batch)로 제공되는) 학습 데이터셋에 대한 예측을 수행하고, 예측 오류를 역전파하여 모델의 매개변수를 조정함

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # 예측 오류 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

모델이 학습하고 있는지를 확인하기 위해 테스트 데이터셋으로 모델의 성능을 확인함

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

학습 단계는 여러번의 반복 단계 (에폭(epochs)) 를 거쳐서 수행된다. 각 에폭에서는 모델은 더 나은 예측을 하기 위해 매개변수를 학습한다. 각 에폭마다 모델의 정확도(accuracy)와 손실(loss)을 출력한다. 에폭마다 정확도가 증가하고 손실이 감소하는 것을 보려고 함

In [ ]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")